In [3]:
from typing import Tuple, Any, Union

from matplotlib import pyplot as plt
from PIL import Image
import numpy as np
import torch

## Help Functions

In [4]:
def show_image(sample: Tuple[Any, Any], transformed_sample: Tuple[Any, Any]):
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))
    titles = ["Original image", "Transformed image"]
    for i, s in enumerate((sample, transformed_sample)):
        axes[i].imshow(s[0], cmap="gray")
        axes[i].set_title(titles[i])
        axes[i].scatter(s[1][0], s[1][1], marker="+", s=100, c="green")

In [5]:
def show_example(sample: Tuple[Image.Image, np.array],
                 prediction: torch.Tensor):
    image, location = sample
    plt.imshow(image.numpy().transpose(1, 2, 0))
    plt.scatter(location[0], location[1], marker="+", s=100, color="green", label="True")
    plt.scatter(prediction[0][0], prediction[0][1], marker="+", s=100, color="red", label="Predicted")
    plt.axis(False)
    plt.legend()

## Pipeline

In [1]:
import sys
sys.path.append('..')

In [2]:
from src.vggregressor.pipeline import Pipeline
from src.config import Config

In [3]:
config = Config.parse("../config_example.yaml")

In [4]:
config.dict()

{'pipeline': {'device': 'cuda', 'epochs': 25},
 'data': {'train': {'path': '../data/processed/localization/train',
   'annotations': 'location.csv',
   'batch_size': 8},
  'valid': {'path': '../data/processed/localization/valid',
   'annotations': 'location.csv',
   'batch_size': 1},
  'test': {'path': '../data/processed/localization/test',
   'annotations': 'location.csv',
   'batch_size': 1}},
 'transforms': {'train': [{'transform': 'Crop',
    'params': {'top': 288, 'left': 0, 'height': 2000, 'width': 3200}},
   {'transform': 'Resize', 'params': {'output_size': 512}}],
  'valid': [{'transform': 'Crop',
    'params': {'top': 288, 'left': 0, 'height': 2000, 'width': 3200}},
   {'transform': 'Resize', 'params': {'output_size': 512}}],
  'test': [{'transform': 'Crop',
    'params': {'top': 288, 'left': 0, 'height': 2000, 'width': 3200}},
   {'transform': 'Resize', 'params': {'output_size': 512}}]},
 'model': {'name': 'VGGRegressor', 'params': {'in_channels': 3}},
 'loss': {'type': 'MSEL

In [5]:
model = Pipeline._create_model(config.model)

In [6]:
data_loaders = Pipeline._create_data_loaders(config.data, config.transforms)

In [7]:
loss = Pipeline._create_loss(config.loss)

In [9]:
optimizer = Pipeline._create_optimizer(config.optimizer, model)

In [11]:
scheduler = Pipeline._create_scheduler(config.lr_scheduler, optimizer)